# Header

In [ ]:
from time import time

# Data manipulation
import pandas as pd

# Metrics 
from sklearn.metrics import accuracy_score, f1_score

In [1]:
from data_treatment import train_df, test_df, \
                            treated_train_df, treated_test_df, \
                            new_features_train_df, new_features_test_df, \
                            svd_train_df, svd_test_df, \
                            targets_for_test_df

from data_treatment import train_val_split                            

# Arima for close price

## Growth factor

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

start_time = time()
X_train, y_train, X_val, y_val = train_val_split(train_df)
close_train_series = X_train['close']
close_val_series = X_val['close']

close_train_growth = close_train_series.pct_change()
close_val_growth = close_val_series.pct_change()

close_train_growth = close_train_growth.dropna()
close_val_growth = close_val_growth.dropna()

# Fit ARIMA model on close_train_growth
arima_model = ARIMA(close_train_growth, order=(5, 1, 0))
arima_result = arima_model.fit()

print(arima_result.summary())

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')

C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

                               SARIMAX Results                                
Dep. Variable:                  close   No. Observations:              1697949
Model:                 ARIMA(5, 1, 0)   Log Likelihood             8181451.130
Date:                Wed, 30 Oct 2024   AIC                      -16362890.261
Time:                        14:11:28   BIC                      -16362816.191
Sample:                             0   HQIC                     -16362870.300
                            - 1697949                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.8442      0.000  -7682.262      0.000      -0.844      -0.844
ar.L2         -0.6877      0.000  -4450.548      0.000      -0.688      -0.687
ar.L3         -0.5079      0.000  -2860.654      0.0

In [ ]:
# Forecast the growth prices using the ARIMA model
forecast_steps = len(close_val_growth)
forecasted_growth = arima_result.forecast(steps=forecast_steps)

forecasted_target = forecasted_growth >= 0

# Print the forecasted growth prices
print(forecasted_growth)

NameError: name 'close_val_growth' is not defined

In [ ]:

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)])	
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)], average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

In [ ]:
# Initialize a list to store the predictions
predicted_growth = []

# Use the last observed value from the training set as the initial input
input_series = close_train_growth.copy()

# Iteratively predict the next value in the validation set
for actual_value in close_val_growth:
    
    # Forecast the next value
    next_forecast = arima_result.forecast(steps=1)[0]
    predicted_growth.append(next_forecast)
    
    # Update the input series with the actual value from the validation set
    input_series = input_series.append(pd.Series([actual_value]))

predicted_target = predicted_growth >= 0
# Print the predicted growth values
print(predicted_growth)
print(predicted_target)


In [ ]:

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)])	
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)], average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

## Price Diff